Q1.  threshold at 2 accuracy = 0.6531

Q2. threshold at 1.5 accuracy =  0.659

Q3. Jaccard threshold at 0.011 accuracy = 0.6187

Q4. Jaccard threshold at 0.008 with popularity threshold at 1.4747 accuracy = 0.67075

Q5. Kaggle user name: Junyan Liu

Q9. lambda = 1 MSE =  1.4907800691966389

Q10. largest user biases (0.00040413237874470305, 'u92864068')
       
       smallest user biases (-0.0015796730337471908, 'u11591742')
       
       largest book biases (0.0008292191795822705, 'b76915592')
       
       smallest book biases (-0.0002721486787445039, 'b57299824')

Q11. lambda = 0.00008 MSE =  1.112277594252111

In [37]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
import numpy
import random
import csv

path = "/Users/apple/Desktop/CSE258/HW3/assignment1/train_Interactions.csv.gz"
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l.strip().split(',')

dataset = []
books = set()
userRatings = defaultdict(set)
userReadings = defaultdict(set)
bookUsersT = defaultdict(set)
validSet = []
for user, book, r in readCSV(path):
    r = int(r)
    books.add(book)
    userRatings[user].add(r)
    userReadings[user].add(book)
    dataset.append([user, book, r])
train = dataset[:190000]
valid = dataset[190000:]
validR = dataset[190000:]

correct = 0
users = set()
allRatings = []
userRatings = defaultdict(set)
bookRatings = defaultdict(set)
userBooks = defaultdict(set)
bookUsers = defaultdict(set)
for item in train:
    u, b, r = item[0], item[1], item[2]
    userBooks[u].add(b)
    bookUsers[b].add(u)
    allRatings.append(r)
    userRatings[u].add(r)
    bookRatings[b].add(r)
    
print(len(bookRatings[2]))

0


In [7]:
negValidSet = []
for item in valid:
    u = item[0]
    b = random.choice(tuple(books))
    while b in userBooks[item[0]]:
        b = random.choice(tuple(books))
    negValidSet.append([u, b, -1])
valid = valid + negValidSet
random.shuffle(valid)

In [8]:
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV(path):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
correct = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break
        
for i in range(len(valid)):
    if (valid[i][1] in return1 and valid[i][2] > -1) or (valid[i][1] not in return1 and valid[i][2] == -1) :
            correct = correct + 1
accuracy = correct/len(valid) 
print("threshold 2 ",accuracy)

return1 = set()
count = 0
correct = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/1.5: break
for i in range(len(valid)):
    if (valid[i][1] in return1 and valid[i][2] > -1) or (valid[i][1] not in return1 and valid[i][2] == -1) :
            correct = correct + 1
accuracy = correct/len(valid) 
print("threshold 1.5 ",accuracy)

threshold 2  0.6479
threshold 1.5  0.6501


In [9]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

predictionsJaccard = [] 
thresholdJaccard = 0.011

correct = 0
def mostSimilar(u, b):
    for b2 in userBooks[u]:
        if b == b2:
            return 1
        sim = Jaccard(bookUsers[b], bookUsers[b2])
        if sim >= thresholdJaccard:
            return 1
    return 0



for i in range(len(valid)):
    if (mostSimilar(valid[i][0], valid[i][1]) == 1 and valid[i][2]>=0) or (mostSimilar(valid[i][0], valid[i][1]) == 0 and valid[i][2] == -1):
        correct = correct + 1
accuracy = correct / len(valid)
print("Jaccard performance ",accuracy)

Jaccard performance  0.62115


In [41]:
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV(path):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return2 = set()
correct = 0
count = 0
for ic, i in mostPopular:
    count += ic
    return2.add(i)
    if count > int(totalRead/1.4747): break
        
thresholdJaccard = 0.0425
def mostSimilar(u, b):
    for b2 in userBooks[u]:
        if b == b2:
            return 1
        sim1 = Jaccard(bookUsers[b], bookUsers[b2])
        for u2 in bookUsers[b]:
            sim2 = Jaccard(userBooks[u], userBooks[u2])
            if sim1 + sim2 >= thresholdJaccard:
                return 1
    return 0

correct = 0
for i in range(len(valid)):
    if ((valid[i][1] in return2 and mostSimilar(valid[i][0], valid[i][1]) == 1) and valid[i][2] > -1) or ( not(valid[i][1]  in return2 and mostSimilar(valid[i][0], valid[i][1]) == 1) and valid[i][2] == -1):
        correct = correct + 1
accuracy = correct/len(valid)
print("Jaccard with popularity ",accuracy)

Jaccard with popularity  0.5688


0.6712 0.04
0.043 

In [42]:
correct = 0
total = 0
p = open('predictions_Read.txt', 'w') 
for i in open("/Users/apple/Desktop/CSE258/HW3/assignment1/pairs_Read.txt"):
    if i.startswith("userID"):
        p.write(i)
        continue
    u, b = i.strip().split('-')
    if (b in return2 and mostSimilar(u, b) == 1):
        s = str(u) + '-' + str(b)+',1\n'
        p.write(s)
    else:
        s = str(u) + '-' + str(b)+',0\n'
        p.write(s)
p.close()

# 9

In [67]:
dataset = []

for user, book, r in readCSV(path):
    r = int(r)
    dataset.append([user, book, r])

train = dataset[:190000]
validR = dataset[190000:]

userBooks = defaultdict(set)
bookUsers = defaultdict(set)
userRatings = defaultdict(set)
bookRatings = defaultdict(set)
for item in train:
    u, b, r = item[0], item[1], item[2]
    userBooks[u].add(b)
    bookUsers[b].add(u)
    userRatings[u].add(r)
    bookRatings[b].add(r)



In [75]:
#complete
def prediction(user, item):
    if user not in userBiases:
        return alpha + itemBiases[item] + inner(userGamma[user], itemGamma[item])
    if item not in itemBiases:
        return alpha + userBiases[user] + inner(userGamma[user], itemGamma[item])
    if user not in userBiases and item not in itemBiases:
        return alpha + inner(userGamma[user], itemGamma[item])
    return alpha + userBiases[user] + itemBiases[item] + inner(userGamma[user], itemGamma[item])

In [69]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

#simple
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:usersN+1]))
    itemBiases = dict(zip(items, theta[1+usersN:]))

In [70]:
#complete
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    alpha = theta[index]
    index += 1
    userBiases = dict(zip(users, theta[index:usersN+index]))
    index += usersN
    itemBiases = dict(zip(items, theta[index:index + itemsN]))
    index += itemsN
    for u in users:
        userGamma[u] = theta[index:index + K]
    for i in items:
        itemGamma[i] = theta[index:index + K]
        index += K

In [71]:
#complete
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])

In [72]:
#complete
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d[0], d[1]) for d in train]
    cost = MSE(predictions, labels)
    #print("MSE test = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
        for k in range(K):
            cost += lamb*userGamma[u][k]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
        for k in range(K):
            cost += lamb*itemGamma[i][k]**2
    return cost

In [73]:
#complete
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(train)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in userRatings:
        dUserGamma[u] = [0.0 for k in range(K)]
    for b in bookRatings:
        dItemGamma[b] = [0.0 for k in range(K)]
    for d in train:
        u,i = d[0], d[1]
        pred = prediction(u, i)
        diff = pred - d[2]
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
        for k in range(K):
            dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
            dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
        for k in range(K):
            dUserGamma[u][k] += 2*lamb*userGamma[u][k]        
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
        for k in range(K):
            dItemGamma[i][k] += 2*lamb*itemGamma[i][k]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return numpy.array(dtheta) 

In [76]:
#complete

import scipy
import scipy.optimize


userGamma = {}
itemGamma = {}
K = 0
globalAverage = float(sum(allRatings)) / len(allRatings)
userAverage = {}
bookAverage = {}
for u in userRatings:
    userAverage[u] = float(sum(userRatings[u])) / len(userRatings[u])
for b in bookRatings:
    bookAverage[b] = float(sum(bookRatings[b])) / len(bookRatings[b])
    
for u in userRatings:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]
for b in bookRatings:
    itemGamma[b] = [random.random() * 0.1 - 0.05 for k in range(K)]

N = len(train)
usersN = len(userRatings)
itemsN = len(bookRatings)
users = list(userRatings.keys())
items = list(bookRatings.keys())
alpha = globalAverage
userBiases = defaultdict(float)
itemBiases = defaultdict(float)
labels = [x[2] for x in train]

scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + # Initialize alpha
                                   [0.0]*(usersN+itemsN) + # Initialize beta
                                   [random.random() * 0.1 - 0.05 for k in range(K*(usersN+itemsN))], # Gamma
                             derivative, args = (labels, 0.000009))

predictionsRatings = []
labels = [x[2] for x in validR]    
for l in validR:
    u,b = l[0], l[1]
    predictionsRatings.append(prediction(u, b))
print("MSE = ", MSE(predictionsRatings, labels))

KeyError: 'b21479253'

In [63]:
MSE =  1.3981046685109149 0.001

0.0000
0.00001  MSE =  1.1103306595248756
0.000009 MSE =  1.1111025838530086

0.0001 MSE =  1.177339418316401
0.000001 MSE =  1.1259275772649437

SyntaxError: invalid syntax (<ipython-input-63-ad0b08fd9975>, line 1)

In [64]:
p = open('predictions_Ratings.txt', 'w') 
total = 0
for i in open("/Users/apple/Desktop/CSE258/HW3/assignment1/pairs_Rating.txt"):
    if i.startswith("userID"):
        p.write(i)
        continue
    u, b = i.strip().split('-')
    p.write(u + '-' + b + ',' + str(prediction(u, b)) + '\n')
    total = total + 1
p.close()

In [31]:
max_user = max(zip(userBiases.values(), userBiases.keys()))
min_user = min(zip(userBiases.values(), userBiases.keys()))
max_book = max(zip(itemBiases.values(), itemBiases.keys()))
min_book = min(zip(itemBiases.values(), itemBiases.keys()))
print("largest user biases",max_user) 
print("smallest user biases",min_user)
print("largest book biases",max_book)
print("smallest book biases",min_book)

largest user biases (1.189854343687313, 'u81539151')
smallest user biases (-3.46140076316382, 'u76571258')
largest book biases (1.0729518667954556, 'b19925500')
smallest book biases (-1.392093899231559, 'b84091840')


In [ ]:
MSE =  1.108031456731227 0.0000164

In [ ]:
MSE =  1.1080406170548083 0.00001631

In [ ]:
MSE =  1.1080445762308229 165

In [ ]:
MSE =  1.1080405294805926  00001638